# Phase Diagram of HK and HKL Model in Two Dimensions

## 1. Find $\mu(\rho)$

### Setup

In [ ]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from scipy.optimize import root_scalar

t = 1
d = 2

def I_1(x):
    if np.abs(x) <= 2*t:
        result = np.heaviside(x + 2*t, 1) - (1 / np.pi) * np.arccos(x / (2*t))
    else:
        result = np.heaviside(x + 2*t, 1)
    return result

def I_2(x):
    def I_1_shifted(k):
        return I_1(x + 2*t*np.cos(k))
    
    integral_value = integrate.quad(I_1_shifted, -np.pi, np.pi)
    
    return integral_value[0] / (2*np.pi)

def rho_hk_2d(mu, U):
    if U >= 0:
        return I_2(mu) + I_2(mu - U)
    elif U < 0:
        return 2 * I_2(mu - U/2)

def find_mu_of_rho_hk(rho, U):
    bracket = (-2*t*d, U + 2*t*d) # minimal and maximal values of mu
    func_mu = lambda mu: rho_hk_2d(mu, U) - rho
    result = root_scalar(func_mu, bracket=bracket, method='brentq')

    if result.converged:
        return result.root
    else:
        raise RuntimeError(f"Keine Nullstelle gefunden für rho={rho}")

def create_mu_list(rho_array, U):
    mu_values = []
    for rho_i in rho_array:
        # print(rho_i)
        try:
            mu = find_mu_of_rho_hk(rho_i, U)
            mu_values.append(mu)
        except RuntimeError:
            mu_values.append(np.nan)
            print(f"Keine Nullstelle gefunden für rho={rho_i}")
    return mu_values

def one_plot(x_array, y_array,x_label, y_label, title):
    plt.figure(dpi=100)
    # Plot erstellen
    plt.plot(x_array, y_array, linestyle='-')
    
    # Achsenbeschriftungen und Titel
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(True)
    plt.title(title)
    plt.show()

#### Testing Setup

In [ ]:
rho_values = np.linspace(0, 2, 500, endpoint=False)

mu_values = create_mu_list(rho_values, 14)

one_plot(rho_values, mu_values, '','','')

## 2. Find $U_c(\rho)$

In [ ]:
def find_U_c_of_rho(rho):
    if rho <= 1:
        func_U = lambda U: find_mu_of_rho_hk(rho, U) + 2*t - U
        result = root_scalar(func_U, bracket=(0, 2*t*d), method='bisect')
    elif rho > 1:
        func_U = lambda U: find_mu_of_rho_hk(rho, U) - 2*t
        result = root_scalar(func_U, bracket=(0, 2*t*d), method='bisect')

    if result.converged:
        return result.root
    else:
        print(result.flag)
        raise RuntimeError(f"Keine Nullstelle gefunden für rho={rho}")
    


    

